In [7]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing import image
from sklearn.preprocessing import LabelEncoder

# Load and preprocess image
def load_image(image_url):
    img = image.load_img(image_url, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    return img

# CNN model for image feature extraction
def get_image_features(image_url):
    model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    img = load_image(image_url)
    features = model.predict(img)
    return features

# Neural network model
def build_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(image_features_size + group_id_size + entity_name_size,)),
        Dense(32, activation='relu'),
        Dense(1)  # Predicting the entity_value
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Preprocessing and encoding
label_encoder = LabelEncoder()

# Example of usage
image_features = get_image_features("https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg")
encoded_group_id = label_encoder.fit_transform([group_id])
encoded_entity_name = label_encoder.fit_transform([entity_name])

# Train the model
# model = build_model()
# model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))



/tmp/ipykernel_5888/2153714741.py:14: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize((image.width * 2, image.height * 2), Image.ANTIALIAS)


Extracted Text from the Image:
ENJOY SLEDDING FUN
WITH YOUR FAMILIES&FRIENDS

 



In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel


2024-09-15 11:38:20.377960: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-15 11:38:20.510241: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 11:38:20.570634: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 11:38:20.586490: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-15 11:38:20.685519: I tensorflow/core/platform/cpu_feature_guar

RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [3]:
train_df = pd.read_csv('sample_train.csv')
test_df = pd.read_csv('sample_test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'sample_train.csv'

In [22]:
print(tf.__version__)

2.17.0


In [1]:
import requests
from PIL import Image
from io import BytesIO
import numpy as np

# Preprocess the images with error handling
def preprocess_images(image_links):
    images = []
    for i, link in enumerate(image_links):
        try:
            response = requests.get(link)
            if response.status_code == 200:  # Check if the request was successful
                img = Image.open(BytesIO(response.content))
                img = img.resize((224, 224))  # Resize images to (224, 224)
                img = np.array(img) / 255.0  # Normalize pixel values
                images.append(img)
            else:
                print(f"Warning: Unable to fetch image at index {i}, URL: {link}")
        except Exception as e:
            print(f"Error: Unable to process image at index {i}, URL: {link}. Error: {str(e)}")
            # Optionally, append a placeholder image or skip this entry
            images.append(np.zeros((224, 224, 3)))  # Placeholder black image
    return np.array(images)

# Apply preprocessing to the training data
X_train_images = preprocess_images(train_df['image_link'].values)

# Check the shape of the resulting images
print(f"Preprocessed images shape: {X_train_images.shape}")


NameError: name 'train_df' is not defined

In [20]:
def get_image_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    return Model(inputs=base_model.input, outputs=x)

In [29]:
# Load BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# # Define an input layer to build the model
# input_ids = tf.keras.Input(shape=(128,), dtype=tf.int32, name="input_ids")
# attention_mask = tf.keras.Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# bert_output = bert_model(input_ids, attention_mask=attention_mask)[0]

from transformers import TFBertModel, BertTokenizer, BertConfig

# Load BERT tokenizer and model configuration
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
config = BertConfig.from_pretrained('bert-base-uncased')

# Load the BERT model
bert_model = TFBertModel(config)

def preprocess_text(texts, max_length=128):
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
    return encodings['input_ids'], encodings['attention_mask']

# Text model for BERT
def get_text_model():
    input_ids = Input(shape=(None,), dtype=tf.int32)
    attention_mask = Input(shape=(None,), dtype=tf.int32)
    bert_output = bert_model([input_ids, attention_mask])[1]  # [1] for pooled output
    x = Dense(64, activation='relu')(bert_output)
    return Model(inputs=[input_ids, attention_mask], outputs=x)

In [30]:
# Categorical embedding model
def get_categorical_model(input_shape):
    inputs = Input(shape=(input_shape,))
    x = Dense(32, activation='relu')(inputs)
    x = Dense(16, activation='relu')(x)
    return Model(inputs=inputs, outputs=x)

In [31]:
def build_model_with_transformer(image_shape, group_shape, entity_shape, text_shape):
    # Image feature extraction
    image_input = Input(shape=image_shape)
    image_model = get_image_model()
    image_features = image_model(image_input)

    # Text model
    text_input_ids = Input(shape=(None,), dtype=tf.int32)
    text_attention_mask = Input(shape=(None,), dtype=tf.int32)
    text_model = get_text_model()
    text_features = text_model([text_input_ids, text_attention_mask])

    # Categorical models
    group_input = Input(shape=(group_shape,))
    entity_input = Input(shape=(entity_shape,))
    
    group_model = get_categorical_model(group_shape)
    entity_model = get_categorical_model(entity_shape)
    
    group_features = group_model(group_input)
    entity_features = entity_model(entity_input)

    # Concatenate all features
    concatenated_features = Concatenate()([image_features, text_features, group_features, entity_features])

    # Dense layers for regression
    x = Dense(128, activation='relu')(concatenated_features)
    x = Dense(64, activation='relu')(x)
    output = Dense(1)(x)  # Predicting entity_value

    model = Model(inputs=[image_input, text_input_ids, text_attention_mask, group_input, entity_input], outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

In [32]:
# Encode categorical variables
def encode_categorical(df):
    group_encoder = LabelEncoder()
    entity_encoder = LabelEncoder()

    df['group_id'] = group_encoder.fit_transform(df['group_id'])
    df['entity_name'] = entity_encoder.fit_transform(df['entity_name'])

    return df, group_encoder, entity_encoder

In [33]:
# Build and train the model
image_shape = (224, 224, 3)  # Image dimensions
group_shape = X_train_groups.shape[1]
entity_shape = X_train_entities.shape[1]
text_shape = (128,)  # Assuming max length for text

model = build_model_with_transformer(image_shape, group_shape, entity_shape, text_shape)
model.summary()

# Example text data preprocessing
texts = ["Example product description here."]
text_input_ids, text_attention_mask = preprocess_text(texts)

# Training
history = model.fit(
    [X_train_images, text_input_ids, text_attention_mask, X_train_groups, X_train_entities],
    y_train,
    epochs=10,
    batch_size=32,
    validation_data=([X_val_images, text_input_ids, text_attention_mask, X_val_groups, X_val_entities], y_val)
)


NameError: name 'X_train_groups' is not defined

In [35]:
# Predict with new data
def predict_with_transformer(image_path, text, group_id, entity_name):
    img_features = preprocess_image(image_path)
    text_input_ids, text_attention_mask = preprocess_text([text])
    group_encoded = group_encoder.transform([group_id]).reshape(-1, 1)
    entity_encoded = entity_encoder.transform([entity_name]).reshape(-1, 1)

    prediction = model.predict([img_features, text_input_ids, text_attention_mask, group_encoded, entity_encoded])
    return prediction[0][0]

# Example usage
print(predict_with_transformer("https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg", "Example product description", "748919", "item_weight"))


FileNotFoundError: [Errno 2] No such file or directory: 'https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg'

In [36]:
def load_data(df):
    images = []
    groups = []
    entities = []
    values = []

    for _, row in df.iterrows():
        # Preprocess image
        img_features = preprocess_image(row['image_link'])
        images.append(img_features.flatten())  # Flatten to fit in dense layers

        # Encode categorical data
        groups.append(row['group_id'])
        entities.append(row['entity_name'])
        
        # Value (for training data)
        if 'entity_value' in row:
            values.append(row['entity_value'])

    if len(values) > 0:
        return np.array(images), np.array(groups), np.array(entities), np.array(values)
    else:
        return np.array(images), np.array(groups), np.array(entities)

# Prepare train data
train_df, group_encoder, entity_encoder = encode_categorical(train_df)
X_train_images, X_train_groups, X_train_entities, y_train = load_data(train_df)

# Prepare test data
test_df, _, _ = encode_categorical(test_df)
X_test_images, X_test_groups, X_test_entities = load_data(test_df)

# Build and train the model
image_shape = (224, 224, 3)  # Image dimensions
group_shape = X_train_groups.shape[1]
entity_shape = X_train_entities.shape[1]
text_shape = (128,)  # Assuming max length for text

model = build_model_with_transformer(image_shape, group_shape, entity_shape, text_shape)
model.summary()

# Example text data preprocessing
texts_train = train_df['text_column'].tolist()  # Adjust the column name accordingly
texts_test = test_df['text_column'].tolist()

text_input_ids_train, text_attention_mask_train = preprocess_text(texts_train)
text_input_ids_test, text_attention_mask_test = preprocess_text(texts_test)

# Train the model
history = model.fit(
    [X_train_images, text_input_ids_train, text_attention_mask_train, X_train_groups, X_train_entities],
    y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2
)

# Predict on test data
predictions = model.predict([X_test_images, text_input_ids_test, text_attention_mask_test, X_test_groups, X_test_entities])

# Prepare output in the required format
output_df = pd.DataFrame({'index': test_df.index, 'prediction': predictions.flatten()})
output_df.to_csv('sample_test_out.csv', index=False)

# Example of how to display predictions with units (modify as necessary)
output_df['prediction'] = output_df['prediction'].apply(lambda x: f"{x:.2f} foot")
print(output_df.head())

# Save the output
output_df.to_csv('sample_test_out.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg'